## Исследование надёжности кредитных заемщиков

Исследование проводится для кредитного отдела банка. Необходимо выяснить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — системы оценки способности возможного заёмщика вернуть кредит.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv('/datasets/data.csv')


data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Данные включают в себя 12 столбцов и 21525 строк. 
Описание столбцов:
<br>children — количество детей в семье, тип - целочисленный
<br>days_employed — общий трудовой стаж в днях, тип - вещественный
<br>dob_years — возраст клиента в годах, тип - целочисленный
<br>education — уровень образования клиента, тип - строковый
<br>education_id — идентификатор уровня образования, тип - целочисленный
<br>family_status — семейное положение, тип - строковый
<br>family_status_id — идентификатор семейного положения, тип - целочисленный
<br>gender — пол клиента, тип - строковый
<br>income_type — тип занятости, тип - строковый
<br>debt — имел ли задолженность по возврату кредитов, , тип - целочисленный
<br>total_income — ежемесячный доход, тип - вещественный
<br>purpose — цель получения кредита, тип - строковый

В столбцах days_employed и total_income есть пропущенные значения, в следующем шаге заполним эти значения.


### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
data['days_employed'].head()

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
Name: days_employed, dtype: float64

В столбце days_employed есть отрицательные значения, которые по смыслу не могут существовать, посмотрим что представляют из себя строки с этими значениями.

In [4]:
data[data['days_employed']<0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья


In [5]:
data[data['days_employed']>0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


In [6]:
print(data[data['days_employed']>0]['days_employed'].max()) #Посмотрим минимальные и максимальные значения положительной части трудового стажа
print(data[data['days_employed']>0]['days_employed'].min())

401755.40047533
328728.72060451825


In [7]:
data[data['days_employed']<0]['days_employed'].abs().max()

18388.949900568383

Значения days_employed>0 и days_employed<0 отличаются многократно, посмотрим в чем причина и можно ли это исправить

In [8]:
data[data['days_employed']>0]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [9]:
data[data['days_employed']<0]['income_type'].value_counts()

сотрудник          10014
компаньон           4577
госслужащий         1312
в декрете              1
предприниматель        1
студент                1
Name: income_type, dtype: int64

In [10]:
print(data[data['days_employed']<0]['days_employed'].mean())
print(data[data['days_employed']>0]['days_employed'].mean())

-2353.0159319988766
365004.3099162686


Все положительные значения относятся только к пенсионерам и безработным и имеют большие значения, которые не соответствуют описанию данных столбца. Все отрицательные значения имеют интерпретируемые значения и относятся ко всем иным типам занятости.
<br>В рамках проекта предположим, что данные трудового стажа по пенсионерам и безработным были записаны в часах и исходя из этого заменим их дни. Столбец в котором, где data_employed<0, приведем к положительным значениям при помощи метода .abs()


In [11]:
data.loc[data['days_employed']>0,'days_employed']=data.loc[data['days_employed']>0,'days_employed']/24

In [12]:
data.loc[data['days_employed']<0,'days_employed']=data.loc[data['days_employed']<0,'days_employed'].abs()

Мы произвели замену значений days_employed, сейчас они все отвечают логике столбца.
Теперь посмотрим нет ли значений, в которых возраст равен нулю.

In [13]:
data[data['dob_years']==0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,14439.234121,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,16577.356876,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19829,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье
20462,0,14113.952856,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,0,13822.552977,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


Данные, в которых возраст равен нулю мы удалим, т.к. нет возможности предположить возраст клиента.
Возможное появления нуля здесь это ошибка ввода либо недостаток информации от клиента.

In [14]:
data=data[data['dob_years']!=0]
data[data['dob_years']==0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [15]:
data[data['days_employed'].isnull()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Обнаружили пропущенные значения в столбцах days_employed и total_income.
<br>Возможные причины этих пропущенных значений - заявка еще в процессе и эти данные клиент должен предоставить со дня на день или у клиента отсутствует как стаж так и доход.

In [16]:
data[data['days_employed'].isnull()]['total_income'].isnull().sum()

2164

In [17]:
data[data['total_income'].isnull()]['days_employed'].isnull().sum()

2164

Все строки, имеющие значение NaN в столбце days_employed, также имеют значение NaN в столбце total_income. 

In [18]:
data[data['total_income']>500000].sort_values(by='total_income',ascending=False)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12412,0,1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью
19606,1,2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости
9169,1,5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование
20809,0,4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1.715018e+06,покупка жилья для семьи
17178,0,5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
4710,1,4247.476347,36,среднее,1,женат / замужем,0,M,сотрудник,0,5.042526e+05,операции со своей недвижимостью
19811,2,3476.970639,31,среднее,1,женат / замужем,0,M,сотрудник,0,5.040793e+05,строительство жилой недвижимости
5414,0,939.533856,51,ВЫСШЕЕ,0,в разводе,3,F,сотрудник,0,5.039504e+05,дополнительное образование
18344,2,1359.583668,40,неоконченное высшее,2,женат / замужем,0,M,сотрудник,0,5.038816e+05,строительство недвижимости


Мы также видим некоторые крупные значения в столбце total_income, однако т.к. нет возможности запросить дополнительные данные(например, профессия или сумма кредита) будем считать информацию, предоставленную в столбце, верной.

Заменим пропущенные значения средними в зависимости от:
<br>а) income_type и education_id для total_income
<br>б) dob_years и education_id для days_employed


#возможно(и наверное даже лучше, учитывая некоторые непонятные крупные значения в total_income) также заменить значения медианами, но я решил, что в рамках проекта не так важно

In [19]:
print(data.groupby('income_type')['total_income'].mean())

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        171008.225815
компаньон          202483.482019
пенсионер          137120.690001
предприниматель    499163.144947
сотрудник          161404.247418
студент             98201.625314
Name: total_income, dtype: float64


Для замены пропущенных значений сгруппируем их по двум необходимым столбцам и применим метод transform() с аргументом lambda x: x.fillna(x.mean()). 
<br>#https://stackoverflow.com/questions/18265930/pandas-filling-missing-values-within-a-group Воспользовался похожей задачей.

<br>Для dob_years также создадим список возможных промежутков.

In [20]:
data.groupby(['income_type','education_id'])['total_income'].mean() #список для заполнения total_income

income_type      education_id
безработный      0               202722.511368
                 1                59956.991984
в декрете        1                53829.130729
госслужащий      0               197612.178874
                 1               154151.857132
                 2               172476.953367
                 3               184056.353037
                 4               111392.231107
компаньон        0               242597.538522
                 1               179575.214672
                 2               197649.335647
                 3               165057.030695
пенсионер        0               170827.674567
                 1               131683.818133
                 2               138312.108423
                 3               111314.924441
                 4               177088.845999
предприниматель  0               499163.144947
сотрудник        0               191703.580506
                 1               152676.027392
                 2            

In [21]:
data['total_income']=data.groupby(['income_type','education_id'])['total_income'].transform(lambda x: x.fillna(x.mean()))

In [22]:
data[data['days_employed'].isnull()]['total_income'].isnull().sum()

0

In [23]:
data.groupby([pd.cut(data['dob_years'],np.arange(0,80,5)),'education_id'])['days_employed'].mean()
#список для заполнения days_employed

dob_years  education_id
(0, 5]     0                        NaN
           1                        NaN
           2                        NaN
           3                        NaN
           4                        NaN
                               ...     
(70, 75]   0               14521.365562
           1               13691.888127
           2               14696.201428
           3               15276.771931
           4                        NaN
Name: days_employed, Length: 75, dtype: float64

In [24]:
data['days_employed']=data.groupby([pd.cut(data['dob_years'],np.arange(0,80,5)),data['education_id']])['days_employed'].transform(lambda x: x.fillna(x.mean()))

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
children            21424 non-null int64
days_employed       21424 non-null float64
dob_years           21424 non-null int64
education           21424 non-null object
education_id        21424 non-null int64
family_status       21424 non-null object
family_status_id    21424 non-null int64
gender              21424 non-null object
income_type         21424 non-null object
debt                21424 non-null int64
total_income        21424 non-null float64
purpose             21424 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Проверим значения остальных переменных.

In [26]:
data['children'].value_counts()

 0     14080
 1      4802
 2      2042
 3       328
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64

Мы видим 47 значений равных -1, чего не может быть согласно логике данных столбца.
<br>Будем считать, что данные клиенты имеют 1 ребенка.
<br>Такая ошибка могла получиться в результате некорректного ввода данных.

<br>Также мы видим 75 клиентов, у которых 20 детей. Столь аномальное количество детей довольно редко, поэтому будем считать это ошибкой ввода и заменим эти значения на 2.

In [27]:
data.loc[data['children']==-1,'children']=1

In [28]:
data.loc[data['children']==20,'children']=2

In [29]:
data['children'].value_counts()

0    14080
1     4849
2     2117
3      328
4       41
5        9
Name: children, dtype: int64

In [30]:
data['gender'].value_counts()

F      14164
M       7259
XNA        1
Name: gender, dtype: int64

Мы видим неизвестное нам значение в поле gender и т.к. оно одно решаем просто избавиться от него.

In [31]:
data=data[data['gender']!='XNA']
data['gender'].value_counts()

F    14164
M     7259
Name: gender, dtype: int64

In [32]:
data['family_status_id'].value_counts()
data['family_status'].value_counts()
data['education'].value_counts()
data['education_id'].value_counts()
data['income_type'].value_counts()
data['debt'].value_counts()
data['purpose'].value_counts()

свадьба                                   792
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     649
операции с жильем                         647
жилье                                     641
покупка жилья                             641
покупка жилья для семьи                   640
строительство собственной недвижимости    633
операции со своей недвижимостью           630
недвижимость                              630
строительство жилой недвижимости          623
покупка недвижимости                      620
строительство недвижимости                620
покупка своего жилья                      619
ремонт жилью                              610
покупка жилой недвижимости                604
на покупку своего автомобиля              502
заняться высшим образованием      

В столбцах, значениях которых мы смотрели ячейкой выше, нет необычных значений. Однако для столца education потребуется удаление дупликатов, а для столбца purpose потребуется лемматизация.
<br> Остальные переменные рассматривались в процессе заполнения пропусков и в них проблем не обнаружено.

In [33]:
data=data.reset_index(drop=True)

### Вывод

В данном разделе были:
<br>удалены значения с нулевым возрастом;
<br>заменены некорректные значения трудового стажа;
<br>заменены некорректные значения трудового дохода;
<br>заменены некорректные значения количества детей;
<br>также была удалена строка с ошибочным полом.

### Замена типа данных

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21423 entries, 0 to 21422
Data columns (total 12 columns):
children            21423 non-null int64
days_employed       21423 non-null float64
dob_years           21423 non-null int64
education           21423 non-null object
education_id        21423 non-null int64
family_status       21423 non-null object
family_status_id    21423 non-null int64
gender              21423 non-null object
income_type         21423 non-null object
debt                21423 non-null int64
total_income        21423 non-null float64
purpose             21423 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Заменим в переменных days_employed и total_income тип данных с вещественного на целочисленный. 
<br>Так как это объекты Series, то воспользуемся методом astype().

In [35]:
try:
    data['days_employed']=data['days_employed'].astype(int)
except:
    print('Возможно в значениях есть текстовое значение')
    

In [36]:
try:
    data['total_income']=data['total_income'].astype(int)
except:
    print('Возможно в значениях есть текстовое значение')

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21423 entries, 0 to 21422
Data columns (total 12 columns):
children            21423 non-null int64
days_employed       21423 non-null int64
dob_years           21423 non-null int64
education           21423 non-null object
education_id        21423 non-null int64
family_status       21423 non-null object
family_status_id    21423 non-null int64
gender              21423 non-null object
income_type         21423 non-null object
debt                21423 non-null int64
total_income        21423 non-null int64
purpose             21423 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

В данном разделе мы заменили тип данных, а также воспользовались конструкцией try-except.

### Обработка дубликатов

В разделе "Предобработка данных" было обнаружено, что в столбце education существуют значения, которые отличаются друг от друга только регистром. Приведем их к единому регистру при помощи метода str.lower().

In [38]:
data['education'].value_counts()

среднее                13691
высшее                  4686
СРЕДНЕЕ                  770
Среднее                  708
неоконченное высшее      665
ВЫСШЕЕ                   273
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [39]:
try:
    data['education']=data['education'].str.lower()
except:
    print('Возможно в столбце есть нестроковый формат')

In [40]:
data['education'].value_counts()

среднее                15169
высшее                  5225
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

Осуществим простой поиск дубликатов строк и удалим их.
<br>Так как в данном массиве неунифицирован столбец purpose, то давайте взглянем на дупликаты без этого столбца.

In [41]:
duplicated_rows=data[data.duplicated(subset=['children','days_employed','dob_years','education_id','family_status_id','gender','income_type','debt','total_income'],keep=False)]
print('Столько строк, в которых значения столбцов за исключением purpose совпадают:',len(duplicated_rows))
duplicated_rows.sort_values(['total_income','days_employed','dob_years']).head(5)          

Столько строк, в которых значения столбцов за исключением purpose совпадают: 1142


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4976,0,3866,50,среднее,1,женат / замужем,0,F,пенсионер,0,131683,на покупку подержанного автомобиля
11113,0,3866,50,среднее,1,женат / замужем,0,F,пенсионер,0,131683,покупка жилья для сдачи
15424,0,3866,50,среднее,1,женат / замужем,0,F,пенсионер,0,131683,покупка жилья для семьи
18469,0,3866,50,среднее,1,женат / замужем,0,F,пенсионер,0,131683,автомобиль
15223,0,6719,51,среднее,1,женат / замужем,0,F,пенсионер,0,131683,покупка жилья для сдачи


In [42]:
data.duplicated(subset=['children','days_employed','dob_years','education_id','family_status_id','gender','income_type','debt','total_income']).sum()


746

Мы видим 1142 строки, в которых есть одинаковые в 11 столбцах строки. Если бы мы решили удалить их, то датасет уменьшился бы на 746 строк. <br>Однако мы не можем их удалить, т.к. заметны разные цели для кредита. Поэтому на данном этапе удалим лишь полные совпадения.

In [43]:
print('Удаляем данное количество строк:',data.duplicated(keep='first').sum())
data[data.duplicated(keep='first')].sort_values(['total_income','days_employed','dob_years']).head()

Удаляем данное количество строк: 71


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21313,0,6719,54,среднее,1,женат / замужем,0,F,пенсионер,0,131683,операции с жильем
9582,0,10136,56,среднее,1,женат / замужем,0,F,пенсионер,0,131683,операции со своей недвижимостью
16119,0,10136,56,среднее,1,женат / замужем,0,F,пенсионер,0,131683,на покупку автомобиля
7771,0,10136,57,среднее,1,гражданский брак,1,F,пенсионер,0,131683,на проведение свадьбы
15194,0,10136,57,среднее,1,гражданский брак,1,F,пенсионер,0,131683,свадьба


In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21423 entries, 0 to 21422
Data columns (total 12 columns):
children            21423 non-null int64
days_employed       21423 non-null int64
dob_years           21423 non-null int64
education           21423 non-null object
education_id        21423 non-null int64
family_status       21423 non-null object
family_status_id    21423 non-null int64
gender              21423 non-null object
income_type         21423 non-null object
debt                21423 non-null int64
total_income        21423 non-null int64
purpose             21423 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [45]:
data=data.drop_duplicates().reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21352 entries, 0 to 21351
Data columns (total 12 columns):
children            21352 non-null int64
days_employed       21352 non-null int64
dob_years           21352 non-null int64
education           21352 non-null object
education_id        21352 non-null int64
family_status       21352 non-null object
family_status_id    21352 non-null int64
gender              21352 non-null object
income_type         21352 non-null object
debt                21352 non-null int64
total_income        21352 non-null int64
purpose             21352 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [46]:
data.isnull().values.any()

False

### Вывод

В данном разделе мы привели к единому виду значения столбца education, убрали полные дупликаты, выявили вероятность появления новых дупликатов после лемматизации столбца purpose.
<br>Для поиска дубликатов в столбце education мы использовали ручной поиск с учетом регистра.
<br>Для поиска дупликата по всем столбцам мы использовали метод duplicated()
<br>Причины появления дупликатов в столбце educated - написание в разных регистрах одних и тех же фраз.
<br>Причины появления общих дупликатов - случайное дублирование заявки.

### Лемматизация

In [47]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

Для лемматизации будем использовать библиотеку pymystem3.
<br>Нам нужно создать список лемм. Для этого создадим одну большую строку, состоящую из всех целей.

In [48]:

list_of_purposes=[]
i=0
for i in range(len(data)):
    word=data['purpose'][i]
    list_of_purposes.append(word)
    i+=1
        
big_string=' '.join(list_of_purposes)


Теперь проведем лемматизацию и посчитаем количество вхождений уникальных лемм при помощи функции Counter().

In [49]:
lemmas=m.lemmatize(big_string)    

In [50]:
print(Counter(lemmas))

Counter({' ': 54785, 'недвижимость': 6327, 'покупка': 5869, 'жилье': 4436, 'автомобиль': 4284, 'образование': 3995, 'с': 2904, 'операция': 2593, 'свадьба': 2310, 'свой': 2223, 'на': 2210, 'строительство': 1873, 'высокий': 1366, 'получение': 1309, 'коммерческий': 1306, 'для': 1286, 'жилой': 1224, 'сделка': 938, 'дополнительный': 902, 'заниматься': 900, 'подержать': 849, 'проведение': 764, 'сыграть': 760, 'сдача': 649, 'семья': 637, 'собственный': 633, 'со': 627, 'ремонт': 605, 'приобретение': 459, 'профильный': 435, 'подержанный': 113, '\n': 1})


При просмотре лемм и значений столбца purpose было решено выделить следующие леммы.

In [51]:
list_of_selected_lemmas=['недвижимость','жилье','автомобиль','свадьба','образование','строительство','ремонт']

In [52]:
def old_add_new_purpose(row):
    new_purposes=m.lemmatize(row['purpose'])
    i=0
    for element in list_of_selected_lemmas:
        if element in new_purposes:
            i+=1
            return element



По результатам работы данной функции выяснилось, что леммы "строительство" и "ремонт" проигнорированы. Для этого было решено изменить функцию так, чтобы вхождение данных лемм также учитывалось.

In [53]:
s_list_of_selected_lemmas=['строительство','ремонт']

In [54]:
def add_new_purpose(row):
    new_purposes=m.lemmatize(row['purpose'])
    for s_element in s_list_of_selected_lemmas:
        if s_element in new_purposes:
            return s_element    
    for element in list_of_selected_lemmas:
        if element in new_purposes:
            return element

           



In [55]:
data['new_purpose'] = data.apply(add_new_purpose,axis=1)

Проведена лемматизация, давайте посмотрим частоту уникальных значений полученного столбца. Также проверим нет ли пропущенных значений.

In [56]:
data['new_purpose'].value_counts()

недвижимость     4454
автомобиль       4284
образование      3995
жилье            3831
свадьба          2310
строительство    1873
ремонт            605
Name: new_purpose, dtype: int64

In [57]:
data['new_purpose'].isnull().sum()


0

In [58]:
data[data['purpose']=='ремонт жилью'].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_purpose
138,0,1765,29,неоконченное высшее,2,женат / замужем,0,M,компаньон,0,196932,ремонт жилью,ремонт
161,0,7473,50,среднее,1,женат / замужем,0,M,сотрудник,0,78975,ремонт жилью,ремонт
226,0,14133,60,среднее,1,Не женат / не замужем,4,F,пенсионер,0,190426,ремонт жилью,ремонт
273,0,8631,64,среднее,1,женат / замужем,0,F,сотрудник,0,160814,ремонт жилью,ремонт
278,1,1912,44,среднее,1,женат / замужем,0,M,компаньон,0,207375,ремонт жилью,ремонт


In [59]:
data[data['purpose']=='строительство собственной недвижимости'].head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_purpose
15,1,972,26,среднее,1,женат / замужем,0,F,сотрудник,0,116820,строительство собственной недвижимости,строительство
27,0,529,28,высшее,0,женат / замужем,0,M,сотрудник,0,308848,строительство собственной недвижимости,строительство
28,1,717,26,высшее,0,женат / замужем,0,F,сотрудник,0,187863,строительство собственной недвижимости,строительство
48,0,3341,45,среднее,1,женат / замужем,0,F,сотрудник,0,162065,строительство собственной недвижимости,строительство
104,0,2098,62,среднее,1,вдовец / вдова,2,F,сотрудник,0,76884,строительство собственной недвижимости,строительство


Заменим названия столбца purpose на old_purpose, а столбца new_purpose на purpose.

In [60]:
data.rename(columns={'purpose':'old_purpose'}, inplace=True)


In [61]:
data.rename(columns={'new_purpose':'purpose'}, inplace=True)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,old_purpose,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


Сохраним отдельно столбец old_purpose, чтобы при случае необходимости мы могли обратиться к нему. Из основного датасета удалим его.

In [62]:
old_purposes=data['old_purpose'].to_dict()

In [63]:
del data['old_purpose']

### Вывод

В данном разделе мы провели лемматизацию столбца purpose, теперь мы можем проанализировать имеющийся датасет более точно.
<br>В ходе работы мы использовались библиотеку pymystem3, а для замены старых значений написали функцию.

### Категоризация данных

Чтобы не перегружать основной датасет создадим словари для столбцов education, family_status.
<br>В основном датасете же оставим только айди этих словарей.
<br>Также категоризируем клиентов по уровню доходов и количеству детей для более наглядного анализа.

In [64]:
education_dict = data[['education_id', 'education']]
education_dict= education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [65]:
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict= family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Мы составили словари значений столбцов family status и education, поэтому теперь можем избавиться от этих столбцов в основном датасете, оставив лишь их айди.

In [66]:
del data['family_status']
del data['education']

In [67]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,жилье
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,1,0,M,сотрудник,0,145885,жилье
3,3,4124,32,1,0,M,сотрудник,0,267628,образование
4,0,14177,53,1,1,F,пенсионер,0,158616,свадьба


In [68]:
income_quantiles=data['total_income'].quantile([0.25, 0.5, 0.75])
income_quantiles


0.25    107648.25
0.50    152304.00
0.75    198293.50
Name: total_income, dtype: float64

In [69]:
data['total_income'].quantile(0.25)

107648.25

In [70]:
def income_level(row):
    income=row['total_income']
    if income<=107648.25:
        return 'низкий'
    if 107648.25<income<=152304.00:
        return 'средний'
    if 152304.00<income<=198293.50:
        return 'высокий'
    else:
        return 'очень высокий'
            
    

        

In [71]:
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,жилье
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,1,0,M,сотрудник,0,145885,жилье
3,3,4124,32,1,0,M,сотрудник,0,267628,образование
4,0,14177,53,1,1,F,пенсионер,0,158616,свадьба


In [72]:
data['income_level']=data.apply(income_level,axis=1)

In [73]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_level
0,1,8437,42,0,0,F,сотрудник,0,253875,жилье,очень высокий
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль,средний
2,0,5623,33,1,0,M,сотрудник,0,145885,жилье,средний
3,3,4124,32,1,0,M,сотрудник,0,267628,образование,очень высокий
4,0,14177,53,1,1,F,пенсионер,0,158616,свадьба,высокий


In [74]:
data['children'].value_counts()

0    14021
1     4839
2     2114
3      328
4       41
5        9
Name: children, dtype: int64

Мы видим довольно малое количество записей для всех клиентов, у кого более двух детей. <br>По этой причине мы добавим столбец, в котором разобьем клиентов на три категории - "Бездетный", "1 ребенок" и "2 и более ребенка"

In [75]:
def children_status(row):
    children_count=row['children']
    if children_count==0:
        return 'бездетный'
    if children_count==1:
        return '1 ребенок'
    if children_count>=2:
        return '2 и более ребенка'

<font color='blue'> Категоризировать по количеству детей не стоит, это сокращает информативность данных. </font>

In [76]:
data['children_status']=data.apply(children_status,axis=1)

In [77]:
data.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_level,children_status
0,1,8437,42,0,0,F,сотрудник,0,253875,жилье,очень высокий,1 ребенок
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль,средний,1 ребенок
2,0,5623,33,1,0,M,сотрудник,0,145885,жилье,средний,бездетный
3,3,4124,32,1,0,M,сотрудник,0,267628,образование,очень высокий,2 и более ребенка
4,0,14177,53,1,1,F,пенсионер,0,158616,свадьба,высокий,бездетный


### Вывод

В данном разделе мы выделили словари для столбцов education и family_status и этим уменьшили объем информации, хранимой в датасет.
<br>Мы также категоризовали доход клиента. Анализ с учетом этих категорий будет легче интерпретировать.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [78]:
children_grouped=data.groupby(['children_status','debt'])['debt'].count()
children_total=data.groupby(['children_status'])['debt'].count()
debt_cause_children=children_grouped/children_total

In [79]:
debt_cause_children.sort_values()

children_status    debt
бездетный          1       0.075458
1 ребенок          1       0.091341
2 и более ребенка  1       0.093499
                   0       0.906501
1 ребенок          0       0.908659
бездетный          0       0.924542
Name: debt, dtype: float64

### Вывод

Мы можем сказать, что чем больше детей у клиента, тем чаще он столкнется с проблемой возврата кредита, однако незначительно.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [80]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_level,children_status
0,1,8437,42,0,0,F,сотрудник,0,253875,жилье,очень высокий,1 ребенок
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль,средний,1 ребенок
2,0,5623,33,1,0,M,сотрудник,0,145885,жилье,средний,бездетный
3,3,4124,32,1,0,M,сотрудник,0,267628,образование,очень высокий,2 и более ребенка
4,0,14177,53,1,1,F,пенсионер,0,158616,свадьба,высокий,бездетный


In [81]:
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [82]:
debt_total=data.groupby(['family_status_id'])['debt'].count()
debt_grouped=data.groupby(['family_status_id','debt'])['debt'].count()
debt_cause_family=debt_grouped/debt_total

In [83]:
debt_cause_family.round(3).sort_values()

family_status_id  debt
2                 1       0.065
3                 1       0.072
0                 1       0.075
1                 1       0.093
4                 1       0.098
                  0       0.902
1                 0       0.907
0                 0       0.925
3                 0       0.928
2                 0       0.935
Name: debt, dtype: float64

### Вывод

Можно заметить, что доля клиентов, которые имели проблемы с возвратом кредита уменьшается в следующем порядке:
<br>Для незамужних/неженатых клиентов доля равна 0.098
<br>Для живущих в гражданском браке - 0.093
<br>Для живущих в браке - 0.075
<br>Для разведенных - 0.072
<br>Для вдовцов - 0.065
<br>Следовательно, наименее проблемная категория клиентов - вдовец / вдова, наиболее проблемная - Не женат / не замужем

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [84]:
incomelevel_total=data.groupby(['income_level'])['debt'].count()
incomelevel_grouped=data.groupby(['income_level','debt'])['debt'].count()
debt_cause_income=incomelevel_grouped/incomelevel_total

In [85]:
debt_cause_income.sort_values()

income_level   debt
очень высокий  1       0.069876
низкий         1       0.079618
высокий        1       0.086549
средний        1       0.088610
               0       0.911390
высокий        0       0.913451
низкий         0       0.920382
очень высокий  0       0.930124
Name: debt, dtype: float64

### Вывод

Мы видим практически одинаковую долю клиентов, имеющих проблемы с возвратом кредитов, для высокого и среднего уровней дохода.
<br>Самая низкая доля с проблемами выплата - у очень высокого уровня доходов, а на втором месте - низкий уровень дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [86]:
purpose_total=data.groupby(['purpose'])['debt'].count()
purpose_grouped=data.groupby(['purpose','debt'])['debt'].count()
debt_cause_purpose=purpose_grouped/purpose_total

In [87]:
debt_cause_purpose.sort_values()

purpose        debt
ремонт         1       0.057851
жилье          1       0.070739
недвижимость   1       0.073866
строительство  1       0.076882
свадьба        1       0.079654
образование    1       0.092616
автомобиль     1       0.093371
               0       0.906629
образование    0       0.907384
свадьба        0       0.920346
строительство  0       0.923118
недвижимость   0       0.926134
жилье          0       0.929261
ремонт         0       0.942149
Name: debt, dtype: float64

### Вывод

Доля проблемных категорий клиентов, взявших кредит на ремонт, всего  0.057, а для целей образования и покупки автомобиля -  0.092 и 0.093. 

### Шаг 4. Общий вывод

В рамках проекта был изучен датасет кредитного отдела. Мы определили и заполнили пропущенные значения, заменили вещественный тип данных на целочисленный.Были обнаружены и удалены дупликаты.
<br>В рамках лемматизации были выделены леммы для столбца purpose, что позволило получить аналитику по целям получения кредита.
<br>В рамках категоризации мы выделили категории у некоторых столбцов, что сделает чтение датасета более понятным.